In [ ]:
''' 

BIG REFERENCE:
https://www.geeksforgeeks.org/generate-graph-using-dictionary-python/

>>>tracker of sorts<<<
December 23 2021: latest update to this project
September 22 2022: first update to project in 2022 after hiatus of 9 months, switched from dynamically searching for text to user entering wikipedia link
November 24 2022: continued updates; made rough map, switched to Gephi (will use networkx later)
November 25 2022: discovered that the computational part behind the visualizations I'm building are called graph theory... kinda forgot about that
'''

'''
thoughts as I go along

> use some network graph package to visualize connection(s)
> main two pages: page_a, page_b
> lol no
    intermediary links between a and b: 
   a_1_1: list of links 1 page, 1st link from page a
   a_10_20: list of links 10 pages, 20th link from page a
   b_300_35: list of links 300 pages, 35th link from page b
    --> recursively checking if any of the links match 
    don't make dynammically named variables, use a dict: 
https://stackoverflow.com/questions/14819849/create-lists-of-unique-names-in-a-for-loop-in-python
> two main requirements for networkx: nodes and edges
    dictionary of lists, example:
        {[source, destination], [source, destination], [source, destination], [source, destination]}
    recursively loop through the origin node list and find all the pages for each origin, insert into list insert into dictionary - using origin page as source
    take the origins nodes pages as source 
    
    before inserting: 
    check for duplicates in origin and destination node ends and in intermediaries
    check for shared edges
    (or insert first and then delete?)
    
    nodes: 
        develop two lists (origin, destination: 
        combine both lists and remove duplictes
    
    edges: 
        provided from above 
        
    research what type of data structure should be used for the source, destination edge file
    

maybe?
> by building a web app, I'd be constantly making requests to Wikipedia and despite their generous no limit, just be considerate policy, i'd rather not crash their systems
   better undertaking: get ALL wikipedia data, store it in one file, build app from there 

'''

In [196]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [197]:
import wikipediaapi
import pandas as pd
import networkx as nx
from datetime import date
from itertools import zip_longest

In [198]:
#   taking the more straightforward route, user supplies program with 2 wikipedia links themselves
search_node_a = input("enter first wikipedia page name: ")
search_node_b = input("enter second wikipedia page name: ")

'''
enter first wikipedia page name: Miramar High School
enter second wikipedia page name: Afridi Redshirt Rebellion, Indian North West Frontier 1930–1931
'''

enter first wikipedia page name: Miramar High School
enter second wikipedia page name: Afridi Redshirt Rebellion, Indian North West Frontier 1930–1931


'\nenter first wikipedia page name: Miramar High School\nenter second wikipedia page name: Afridi Redshirt Rebellion, Indian North West Frontier 1930–1931\n'

In [199]:
wiki_wiki = wikipediaapi.Wikipedia('en')

page_a = wiki_wiki.page(search_node_a)
page_b = wiki_wiki.page(search_node_b)

In [200]:
def return_links(page, list_n):
        links = page.links
        for title in sorted(links.keys()):
            list_n.append(title)

In [201]:
list_node_a = []
list_node_b = []

return_links(page_a, list_node_a)
return_links(page_b, list_node_b)

In [202]:
def clean_links(list_node):
    for item in list_node.copy():
        if ((item in ('Wayback Machine', 'Specials (Unicode block)')) or (item.endswith(('(disambiguation)', '(identifier)'))) or (item.startswith(('List of ', 'Category:', 'File:', 'Help:', 'Talk:', 'Template:', 'Wikipedia:', 'Template talk:', 'Portal:', 'Wiki')))):
            list_node.remove(item)

In [203]:
clean_links(list_node_a)
clean_links(list_node_b)

In [204]:
graph = {}

#add nodes of searches a, b as values and titles as keys to 'graph' dict
graph[(page_a.title)] = list_node_a
graph[(page_b.title)] = list_node_b

G = nx.to_networkx_graph(graph)

In [11]:
'''

error = ""

try:
    nx.shortest_path(G, page_a.title, page_b.title)
except nx.NetworkXNoPath:
    error = "No direct path found between %s and %s" % (page_a.title, page_b.title)
else:
    shortest_path_list = nx.shortest_path(G, page_a.title, page_b.title)
    
if error == "":
    print("There are " + str(len(shortest_path_list) - 1) + " degrees of separation between " + page_a.title + " and " + page_b.title + "\n")
    print(shortest_path_list)
else:
    print(error)
    
'''

No direct path found between War of 1812 and Lionel Messi


In [205]:
title = []

page_list = [] # TO CLEAR AFTER EVERY ITERATION

title.append(page_a.title)
title.append(page_b.title)

error = ""

In [216]:
#BULK OF QUERY
combined_a_b = [i for si in zip_longest(*graph.values()) for i in si]
title = [i for i in combined_a_b if i is not None]

for i in title:
    try:
        page = wiki_wiki.page(i)
        return_links(page, page_list)
        clean_links(page_list)
        
        if i not in title:
            title.append(i)

        graph[(page.title)] = page_list

        G = nx.to_networkx_graph(graph)
    except RuntimeError:
        raise;
    else:
        page = wiki_wiki.page(i)
        return_links(page, page_list)
        clean_links(page_list)
        
        if i not in title:
            title.append(i)
            continue; 

        graph[(page.title)] = page_list

        G = nx.to_networkx_graph(graph)
    finally:
        try:
            nx.shortest_path(G, page_a.title, page_b.title)
        except nx.NetworkXNoPath:
            error = "No path found between %s and %s" % (page_a.title, page_b.title)
        else:
            shortest_path_list = nx.shortest_path(G, page_a.title, page_b.title)
        finally:
            if error == "":
                print("There are " + str(len(shortest_path_list) - 1) + " degrees of separation between " + page_a.title + " and " + page_b.title + "\n")
                print(shortest_path_list)
                break;
            else:
                print(i)
    
    page_list.clear();
    del page; 
                    

#        if error == "":
#            print("There are " + str(len(shortest_path_list) - 1) + " degrees of separation between " + page_a.title + " and " + page_b.title + "\n")
#            print(shortest_path_list)
#            break;
#        else:
#            #print(error)
#            print(i)
#            #continue;

#        page_list.clear();
#        del page; 

    #else: #value is in the title list
    #    continue;

American Heritage School (Florida)
1936–1939 Arab revolt in Palestine
American InterContinental University South Florida Campus
Action of 17 July 1628
Archbishop Edward A. McCarthy High School
Aden Emergency
Asian people
Aden Expedition
Athletic nickname
Adubi War
Atlantic Technical Center
Adwan Rebellion
Black people
Afridi
Blanche Ely High School
Ahmed Barzani revolt
Boyd H. Anderson High School
Alexandria expedition of 1807
Broward College
Ambela campaign
Broward County
American Revolutionary War
Broward County Public Schools
Anglo-Afghan Wars
Broward County, Florida
Anglo-Aro War
Broward Virtual Education High
Anglo-Ashanti wars
Cardinal Gibbons High School (Fort Lauderdale, Florida)
Anglo-Cherokee War
Chaminade-Madonna College Preparatory School
Anglo-Egyptian War
Charles W. Flanagan High School
Anglo-French blockade of the Río de la Plata
Cincinnati Enquirer
Anglo-Manipur War
City College (Florida)
Anglo-Mughal War
Clarence Coleman (gridiron football)
Anglo-Nepalese War
Coconut C

In [217]:
df = pd.DataFrame(
    [(k, i) for k, v in graph.items() for i in v], 
    columns=['key_id', 'ids']
)

#df.loc[(df["key_id"] == '1916 South American Championship') & (df["ids"] == 'War of 1812')]
df
#title
#page_list

,key_id,ids
0,Miramar High School,American Heritage School (Florida)
1,Miramar High School,American InterContinental University South Flo...
2,Miramar High School,Archbishop Edward A. McCarthy High School
3,Miramar High School,Asian people
4,Miramar High School,Athletic nickname
...,...,...
305,"Afridi Redshirt Rebellion, Indian North West F...",Williamite War in Ireland
306,"Afridi Redshirt Rebellion, Indian North West F...",Woyane rebellion
307,"Afridi Redshirt Rebellion, Indian North West F...",Xhosa Wars
308,"Afridi Redshirt Rebellion, Indian North West F...",Yamasee War
